<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import numpy as np
from numba import jit, jitclass, int64, float64

spec = [('w', int64),
        ('h', int64),
        ('D', int64)]

@jitclass(spec)
class hotplate:
    def __init__(self, w=10, h=20, D=4):
        self.w = w
        self.h = h
        self.D = D

spec = [('boundary_conditions', float64[:]),
        ('relaxation_guess',float64),
        ('ds',float64),
        ('maxiter',int64),
        ('dx', float64),
        ('dy', float64),
        ('dx2', float64),
        ('dy2', float64),
        ('nx', int64),
        ('ny', int64),
        ('dt', float64),
        ('u0', float64[:,:]),
        ('u', float64[:,:]),
        ('D', float64)]
@jitclass(spec)
class steadystate:
    def __init__(self,
                 hotplate,
                 boundary_conditions=[100.,0.,0.,.0],
                 relaxation_guess=30.,
                 ds = 0.1,
                 maxiter=500):
        self.boundary_conditions = boundary_conditions
        self.relaxation_guess = relaxation_guess
        self.ds = ds
        self.maxiter = maxiter
        # self.D = diffusivity of hot plate material
        self.D = hotplate.D
        # SEE: second derivatives of spatially-dependent terms
        self.dx, self.dy = ds, ds
        self.dx2, self.dy2 = (self.dx*self.dx), (self.dy*self.dy)
        self.dt = self.dx2*self.dy2 / (2 * hotplate.D * (self.dx2 + self.dy2))
        # number of discrete points in each cardinal direction
        self.nx, self.ny = int(hotplate.w/self.dx), int(hotplate.h/self.dy)
        # Initialize the array we'll do calculations on
        # and set the interior value to Tguess
        self.u0 = np.empty((self.nx, self.ny), dtype=np.float64)
        for i in range(self.nx):
            for j in range(self.ny):
                self.u0[i, j] = relaxation_guess
        # Set boundary conditions for the steady state before t=0
        self.u0[:, (self.ny-1):] = boundary_conditions[0]
        self.u0[:, :1] = boundary_conditions[1]
        self.u0[:1, :] = boundary_conditions[-2]
        self.u0[(self.nx-1):, :] = boundary_conditions[-1]
        
    def calculate(self):
        # Compute the steady state distribution, where u0 = Steady-State Temperature Matrix
        for iteration in range(0, self.maxiter):
            for i in range(1, self.nx-1):
                for j in range(1, self.ny-1):
                    self.u0[i, j] = (0.25) * (self.u0[i+1][j] + self.u0[i-1][j] +
                                        self.u0[i][j+1] + self.u0[i][j-1])

In [3]:
spec = []
@jitclass(spec)
class hotboundaries:
    def __init__(self,
                 steadyplate, # pass in a hotplate with an initialized and/or state
                 boundary_conds=[0.0,0.0,0.0,0.0]):
        # New initial conditions
        Ttop_new = boundary_conds[0]
        Tbottom_new = boundary_conds[1]
        Tleft_new = boundary_conds[-2]
        Tright_new = boundary_conds[-1]
        # Set new boundary conditions (Temperatures) in the steady-state matrix
        steadyplate.u0[:, :5] = Tbottom_new
        steadyplate.u0[(steadyplate.nx-5):, :] = Tright_new
        steadyplate.u0[:5, :] = Tleft_new
        steadyplate.u0[:, (steadyplate.ny-5):] = Ttop_new

# Hotspot produces a hot spot on the plate as an initial condition of the heatflow
# circle of radius r, centred at c=(cx,cy)
spec = []
@jitclass(spec)
class hotspot:
    def __init__(self,
                 # pass in a hotplate with an initialized and/or steady state
                 steadyplate,
                 # temperature of spot
                 temperature = 200,
                 # radius of the spot
                 r = 1,
                 # position of center of the hotspot
                 c = (0, 0)
                 ): 
        # r2 = radius squared
        r2 = r*r 
        for i in range(steadyplate.nx):
            for j in range(steadyplate.ny):
                p2 = (i * steadyplate.dx - c[0])**2 + (j * steadyplate.dy - c[1])**2
                if p2 < r2:
                    steadyplate.u0[i, j] = temperature

spec = [('tsteps', int64),
        ('D', float64),
        ('dx2', float64),
        ('dy2', float64),
        ('nx', float64),
        ('ny', float64),
        ('dt', float64),
        ('u0', float64[:,:]),
        ('u', float64[:,:]),
        ('frames', float64[:,:,:])]
@jitclass(spec)
class heatflow:    
    def do_timestep(self):
        # Propagate with forward-difference in time, central-difference in space
        self.u[1:-1, 1:-1] = self.u0[1:-1, 1:-1] + self.D * self.dt * (
            (self.u0[2:, 1:-1] - 2*self.u0[1:-1, 1:-1] + self.u0[:-2, 1:-1])/self.dx2
            + (self.u0[1:-1, 2:] - 2*self.u0[1:-1, 1:-1] + self.u0[1:-1, :-2])/self.dy2)
        
        self.u0 = self.u

    def __init__(self,
                 steadyplate # pass in a hotplate entity with an initialized and/or steady state
                ):
        self.u = steadyplate.u0.copy()
        self.u0 = steadyplate.u0
        self.dt = steadyplate.dt
        self.dx2 = steadyplate.dx2
        self.dy2 = steadyplate.dy2
        self.nx = steadyplate.nx
        self.ny = steadyplate.ny
        self.D = steadyplate.D

    def calculate(self,
                  tsteps, # number of time steps
                  snapshots # on which timesteps should snapshots (frames) be save for visualizing?
                  ):
        # frames = timeseries of snapshots from the heatflow (used for visualizing heatflow)
        self.frames = np.empty((int(self.nx), int(self.ny), len(snapshots)), dtype=np.float64)
        
        # index for saving frames on the z-axis if the self.frames
        i = 0
        
        # perform the specified number of timesteps of heatflow
        for n in range(tsteps): 
            self.do_timestep()
            # save the specified frames
            if n in snapshots:
                self.frames[:,:,i] = self.u0.copy()
                i+=1

In [4]:
import sys
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
from heatflow import *
from matplotlib import cm

class  CustomWidget(pg.GraphicsWindow):
    pg.setConfigOption('background', 'k')
    pg.setConfigOption('foreground', 'w')
    ptr1 = 0
    def __init__(self, parent=None, **kargs):
        pg.GraphicsWindow.__init__(self, **kargs)
        self.setParent(parent)
        self.setWindowTitle('Heatflow Animation')

        view = self.addViewBox()
        view.setAspectLocked(True) # so pixels stay square

        self.img = pg.ImageItem(border='w') # this displays the heatflow
        view.addItem(self.img)
        view.setRange(QtCore.QRectF(0, 0, 600, 600)) # initial view bounds
        
        ## Display FPS and iteration count
        self.fps_txt = pg.TextItem(text='FPS: 0.0', color=(200, 200, 200), 
                              anchor=(0, 0), border=None, fill=None)
        self.iter_txt = pg.TextItem(text='Timestep: 0.0', color=(200, 200, 200), 
                               anchor=(0, 0), border=None, fill=None)

        self.fps_txt.setFont(pg.Qt.QtGui.QFont("times", 10))
        self.iter_txt.setFont(pg.Qt.QtGui.QFont("arial", 10))

        self.fps_txt.setPos(0, 675)
        self.iter_txt.setPos(0, 650)

        view.addItem(self.fps_txt)
        view.addItem(self.iter_txt)
        
        ""' BEGIN - USER OPTIONS REPLACE CODE BELOW '""
        ## Generate steady state object and initialize heatflow
        b = steadystate(hotplate(60, 60, 4),
                        boundary_conditions=np.array([100.,0.,0.,.0]),
                        relaxation_guess=30.,
                        ds = 0.1,
                        maxiter=500)
        b.calculate()

        # Set IC's after steady state is reached but before heatflow occurs
        hotboundaries(b, np.array([100.,60.,60.,60.]))
        hotspot(b, 80., 25, (30, 30))
        hotspot(b, 50., 15, (30, 30))
        hotspot(b, 99., 5, (30, 30))
        self.b = heatflow(b)
        ""' END - USER OPTIONS REPLACE CODE ABOVE'""
        
        ## Colormap the values of the heatflow matrix
        colormap = cm.get_cmap("inferno")
        colormap._init()
        # Lookup Table (lut); converts mpl.cm from 0-1 to 0-255 for Qt
        lut = (colormap._lut * 255).view(np.ndarray)
        # Apply the colormap
        self.img.setLookupTable(lut) 
        ## Track number of iterations and fps
        self.updateTime = pg.ptime.time()
        self.fps = 0
        self.i = 0
        # start the heatflow calculation loop (self.update)
        timer = pg.QtCore.QTimer(self)
        timer.timeout.connect(self.update)
        timer.start(0)
        
    def update(self):
        self.b.calculate(1, [0])
        ## Display the data
        self.img.setImage(self.b.u0)
        # set the max an min levels to interpolate
        self.img.setLevels([0,100]) 
        # calculate FPS
        now = pg.ptime.time()
        fps2 = 1.0 / (now - self.updateTime)
        self.updateTime = now
        self.fps = self.fps * 0.9 + fps2 * 0.1
        
        self.fps_txt.setText('FPS: {}'.format(round(self.fps, 2)), (255,0,0))
        self.iter_txt.setText('Timestep: {}'.format(self.i), (255,0,0))
        
        self.i+=1

if __name__ == '__main__':
    w = CustomWidget()
    w.show()
    QtGui.QApplication.instance().exec_()

In [1]:
import sys
import pyqtgraph as pg
from pyqtgraph.Qt import QtWidgets, QtGui, QtCore

QWidget = QtWidgets.QWidget
QApplication = QtWidgets.QApplication
QPushButton = QtWidgets.QPushButton
pyqtSlot = QtCore.pyqtSlot

class App(QWidget):
    def __init__(self):
        super().__init__()
        self.title = 'PyQt5 button'
        self.left = 10
        self.top = 10
        self.width = 320
        self.height = 200
        self.initUI()
    
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        
        button = QPushButton('PyQt5 button', self)
        button.setToolTip('This is an example button')
        button.move(100,70)
        button.clicked.connect(self.on_click)
        
        self.show()

    @pyqtSlot()
    def on_click(self):
        print('button click')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())


PyQt5 button click
PyQt5 button click
PyQt5 button click
PyQt5 button click
PyQt5 button click
PyQt5 button click
PyQt5 button click
PyQt5 button click


SystemExit: 0

C:\Users\allen\Miniconda3\envs\qtdev\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
help(pyqtSlot)

Help on built-in function pyqtSlot in module PyQt5.QtCore:

pyqtSlot(...)
    @pyqtSlot(*types, name: Optional[str], result: Optional[str])
    
    This is a decorator applied to Python methods of a QObject that marks them
    as Qt slots.
    The non-keyword arguments are the types of the slot arguments and each may
    be a Python type object or a string specifying a C++ type.
    name is the name of the slot and defaults to the name of the method.
    result is type of the value returned by the slot.

